# 문서 분류를 하는 자연어 분류기를 만들어보자
- Input : 문서 --> ["현재 금리상태는 ...]
- Output : 시제 --> ["현재, 미래, 과거"]
- Model
    - 글자 Encoding : TF-IDF
    - 학습 모델 : Logistic Regression

## 0. 라이브러리 불러오기

### 1) 기본 라이브러리

In [1]:
import numpy as np
import pandas as pd

### 2) 딥러닝 라이브러리

In [2]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## 1. 데이터 불러오기

In [3]:
df = pd.read_csv("./data/train.csv")

In [4]:
df

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


In [5]:
df = df[:200]

In [6]:
df.head(2)

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실


## 2. Label Indexing 

### 1) 시제 Unique 개수 추출

In [7]:
df["시제"].value_counts()

과거    99
현재    80
미래    21
Name: 시제, dtype: int64

In [8]:
label_list = df["시제"].value_counts().keys().tolist()
num_label = len(label_list)

In [9]:
num_label

3

### 2) 시제 Label Indexing

In [10]:
def label_encoder(x):
    result = 0
    if x == "현재":
        result = 0
    elif x == "과거":
        result = 1
    elif x == "미래":
        result = 2
    else:
        print("error")
    return result

In [11]:
label_encoder("과거")

1

### 3) 데이터 프레임 생성 + 데이터 프레임 합치기 + 열 이름 설정

In [12]:
df["Label_Num"] = df["시제"].apply(lambda x: label_encoder(x))

C:\Users\Bang\AppData\Local\Temp\ipykernel_29596\4214151600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Label_Num"] = df["시제"].apply(lambda x: label_encoder(x))


## 3. 문장수 + TF-IDF 계산

### 1) Sentence 길이 추출 + 데이터 프레임 생성 + 데이터 프레임 합치기 + 열 이름 설정

In [13]:
df["Sentence_Len"] = df["문장"].apply(lambda x: len(x.split(".")))
#df["content"].apply(lambda x: len(x.split("."))).value_counts()

C:\Users\Bang\AppData\Local\Temp\ipykernel_29596\617351959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sentence_Len"] = df["문장"].apply(lambda x: len(x.split(".")))


### 2) TF-IDF 추출 + 데이터 프레임 생성 + 데이터 프레임 합치기 + 열 이름 설정

In [14]:
corpus = df['문장']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
df["Sentence_Vector"] = X.todense().tolist()

C:\Users\Bang\AppData\Local\Temp\ipykernel_29596\4262522289.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sentence_Vector"] = X.todense().tolist()


In [15]:
vectorizer.get_feature_names()

c:\Users\Bang\Anaconda\envs\BangEnv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['075',
 '100',
 '1000개',
 '1000달러가',
 '100은',
 '10년',
 '10만1500원을',
 '10만km',
 '10만개',
 '10명',
 '10번째',
 '10위권',
 '10일',
 '11',
 '11월',
 '1296개를',
 '12만1000명으로',
 '12월',
 '12월중',
 '147억원을',
 '1481곳에서',
 '14일',
 '15',
 '150',
 '15년만에',
 '15만원만',
 '1600',
 '16일',
 '18조9000억원',
 '1917',
 '1928',
 '1950',
 '1963년생이거나',
 '1965년',
 '1970년은',
 '1973년에',
 '1974년에',
 '1988년',
 '1989년',
 '1994년',
 '1995년',
 '19일',
 '1년까지',
 '1년에',
 '1번기가',
 '1번기와',
 '1시부터',
 '1억5000만',
 '1억6000만달러의',
 '1억원의',
 '1월',
 '1월들어',
 '1위는',
 '1위에',
 '1일',
 '1점',
 '1회',
 '20',
 '2000',
 '2001년',
 '2005년부터',
 '2009년',
 '200m급',
 '2012년',
 '2013년',
 '2015년',
 '2016',
 '2016년에는',
 '2018년',
 '2020',
 '2020년',
 '2021년',
 '20만원',
 '20일부터',
 '21',
 '21만4000명에서',
 '21의',
 '229',
 '24시간',
 '25조원',
 '27일',
 '27일에도',
 '280칸으로',
 '28년',
 '2관왕에',
 '2년이',
 '2번과',
 '2월',
 '2위',
 '2종으로',
 '2함대에서',
 '30',
 '3000만명',
 '300만원',
 '30일',
 '30초',
 '32',
 '37',
 '3년간',
 '3번기가',
 '3억8000만',
 '3월',
 '3월부터는',
 '3위까지',
 '3차례',
 '3초',
 '3함대에서',
 '

In [16]:
type(X)

scipy.sparse._csr.csr_matrix

In [17]:
vectorizer

TfidfVectorizer()

In [18]:
print(X.shape)

(200, 2392)


In [19]:
X.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
X.todense()[0].tolist()

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.3574958126745188,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.3574958126745188,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [21]:
X.todense()[0].tolist()[0]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3574958126745188,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3574958126745188,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3574958126745188,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 

In [22]:
df

,ID,문장,유형,극성,시제,확실성,label,Label_Num,Sentence_Len,Sentence_Vector
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...
195,TRAIN_00195,포이 국장은 ＂현재 유럽 데이터센터의 45%가량이 룩셈부르크에 자리 잡고 있다＂고 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
196,TRAIN_00196,엔씨소프트는 새 시즌을 맞아 업데이트 예정인 신규 콘텐츠도 공개했다.,사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
197,TRAIN_00197,특히 이 부회장은 ＂우리의 모든 영화에 대해 주저하지 않고 의견을 바로 말씀해주신 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
198,TRAIN_00198,김치는 지난해 전년 대비 11% 증가한 1억6000만달러의 수출액을 기록했다.,사실형,긍정,과거,확실,사실형-긍정-과거-확실,1,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
df['Sentence_Vector']

0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                             ...                        
195    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
196    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
197    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
198    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
199    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: Sentence_Vector, Length: 200, dtype: object

In [24]:
df['Sentence_Vector'].values.tolist()

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.3574958126745188,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.3574958126745188,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [25]:
torch.tensor(df["Sentence_Vector"].values.tolist()).to(torch.float32)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [26]:
df['Sentence_Len']

0      3
1      2
2      2
3      2
4      2
      ..
195    2
196    2
197    3
198    2
199    2
Name: Sentence_Len, Length: 200, dtype: int64

### 3) TF-IDF 개수 출력

In [27]:
for i in df['Sentence_Vector']:
    print(len(i))

2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392
2392


## 3. 모델 정의 + 학습

### 1) 모델 정의

In [28]:
num_features = len(df['Sentence_Vector'][0])
num_classes = 3

In [29]:
class MultiClassClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(in_features = num_features, out_features = num_classes)
        # self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        return self.linear(x)
        # return self.sigmoid(self.linear(x))

### 2) 데이터 불러오기 + CustomDataset 정의

In [30]:
x_train = torch.tensor(df["Sentence_Vector"].values.tolist()).to(torch.float32)
y_train = torch.tensor(df["Label_Num"].values)

In [31]:
x_train

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [32]:
y_train

tensor([0, 1, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 1, 0, 1, 1, 0, 0, 1, 1,
        2, 1, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 1, 1, 2, 0, 1, 0, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 2, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 2, 1, 2, 0,
        0, 1, 0, 2, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 2, 0, 2,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 2, 2, 1, 1, 0, 0, 1, 2, 0, 1, 1, 0, 1, 2,
        1, 1, 1, 1, 1, 1, 1, 1])

In [33]:
class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.df = df
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = torch.tensor(self.y_data[idx])
        
        return {"x":x, "y":y}

In [34]:
train_dataset = CustomDataset(x_train, y_train)

### 3) DataLoader 정의

In [35]:
train_loader = DataLoader(train_dataset, batch_size = 10)

### 5) 모델 + Optimizer 정의

In [36]:
model = MultiClassClassifier()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [37]:
train_loader_iter = iter(train_loader)
batch_idx, samples = next(train_loader_iter)

C:\Users\Bang\AppData\Local\Temp\ipykernel_29596\322247256.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.y_data[idx])


### 6) 모델 학습

In [38]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, batch in enumerate(train_loader):
        x_train, y_train = batch["x"], batch["y"]
        # (1) 모델 예측
        y_pred = model(x_train)
        print(y_pred, y_train)
        # (2) Cost 계산
        cost = F.cross_entropy(y_pred, y_train)
        
        # (3) Threshold 기준 분류
        # print(y_pred)
        prediction = torch.argmax(y_pred, dim=1)
        # print(prediction, y_train)
        # (4) Accuracy 계산
        check_prediction = prediction == y_train
        accuracy = check_prediction.sum().item() / len(check_prediction)
        
        print('Epoch {:4d}/{} Batch {}/{} Cost:{:.6f} Accruacy:{:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(train_loader), cost.item(), accuracy * 100)) #  hypothesis: {} , pred.squeeze().detach()
        
        # (5) Optimizer 초기화
        optimizer.zero_grad()
        # (6) Cost 계산
        cost.backward()
        # (7) Optimizer 업데이트
        optimizer.step()

C:\Users\Bang\AppData\Local\Temp\ipykernel_29596\322247256.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.y_data[idx])


Epoch    0/20 Batch 1/20 Cost:1.100724 Accruacy:0.000000
Epoch    0/20 Batch 2/20 Cost:1.078508 Accruacy:70.000000
Epoch    0/20 Batch 3/20 Cost:1.094304 Accruacy:10.000000
Epoch    0/20 Batch 4/20 Cost:1.045223 Accruacy:50.000000
Epoch    0/20 Batch 5/20 Cost:1.032662 Accruacy:30.000000
Epoch    0/20 Batch 6/20 Cost:1.060767 Accruacy:30.000000
Epoch    0/20 Batch 7/20 Cost:0.990768 Accruacy:50.000000
Epoch    0/20 Batch 8/20 Cost:0.957798 Accruacy:70.000000
Epoch    0/20 Batch 9/20 Cost:0.968299 Accruacy:50.000000
Epoch    0/20 Batch 10/20 Cost:0.835520 Accruacy:60.000000
Epoch    0/20 Batch 11/20 Cost:0.912572 Accruacy:60.000000
Epoch    0/20 Batch 12/20 Cost:0.938773 Accruacy:30.000000
Epoch    0/20 Batch 13/20 Cost:0.946960 Accruacy:50.000000
Epoch    0/20 Batch 14/20 Cost:0.772484 Accruacy:60.000000
Epoch    0/20 Batch 15/20 Cost:1.373304 Accruacy:20.000000
Epoch    0/20 Batch 16/20 Cost:0.835147 Accruacy:40.000000
Epoch    0/20 Batch 17/20 Cost:1.109671 Accruacy:50.000000
Epoch  